# Libraries

Importing the libraries necessary.

In [ ]:
# Libraries
from simtool import findInstalledSimToolNotebooks, searchForSimTool
from simtool import getSimToolInputs, getSimToolOutputs, Run
#meltheas -> tldr; seperate jupyter notebook with existing code
MeltHEA = searchForSimTool('meltheas')

import json
import pandas as pd
import re
import os
import math
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, HBox
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
from pathlib import Path

import seaborn as sns

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import nanohubremote as nr
from scipy.stats import norm


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE, MDS, LocallyLinearEmbedding, Isomap, SpectralEmbedding

# Querying Data

Interacting with the nanoHub API to query relevant data from the MELTHEAS simtool.

In [ ]:
# CREDENTIALS

auth_data = { 'grant_type' : 'tool' }
with open(os.environ["SESSIONDIR"]+"/resources") as file:
    lines = [line.split(" ", 1) for line in file.readlines()]
    properties = {line[0].strip(): line[1].strip() for line in lines if len(line)==2}
    auth_data["sessiontoken"] = properties["session_token"]
    auth_data["sessionnum"] = properties["sessionid"]
    
session = nr.Tools(auth_data)

#TOOL NAME
tool = 'meltheas' 

# QUERY FOR INPUTS AND OUTPUTS
req_json = session.requestPost('results/dbexplorer/tool_detail?simtool=true', data={'tool': tool})
req_json = req_json.json()
parameters = req_json['results']
inputs = list(parameters[0][tool]['input'].keys())
outputs = list(parameters[0][tool]['output'].keys())


# QUERY EVERYTHING FROM THE DATABASE
search = {
    'tool':tool,
    'filters':json.dumps([
        {'field':'input.Tsolid','operation':'>=','value':'2'},     # Filters included to query [Everything] in the cached database
        {'field':'input.composition1','operation':'<=','value':'0.5'},
        {'field':'input.composition2','operation':'<=','value':'0.5'},
        {'field':'input.composition3','operation':'<=','value':'0.5'},
        {'field':'input.composition4','operation':'<=','value':'0.5'},
        {'field':'input.composition5','operation':'<=','value':'0.5'},        
    ]),
    'results':json.dumps(['input.composition1','input.composition2','input.composition3','input.composition4','input.composition5',
                          'input.Tsolid', 'input.Tliquid',
                          'output.Coexistence', 'output.Converged', 'output.melting_temperature',
                          'output.melting_temperature_ci', 'output.fraction_solid', 'output.fraction_liquid', 'output.counts_array',]),    # Selected Parameters (Inputs/Outputs) that will be requested from the query
    'limit':0,    
    'revision':0,
 }

req_json = session.requestPost('results/dbexplorer/search?simtool=true', data=search, timeout=30)
results = req_json.json()

#RESET INDEX AND DROP NA VALUES
complete_dataframe = pd.DataFrame(results['results']).dropna()
complete_dataframe = complete_dataframe.reset_index(drop=True)     

In [ ]:
#RENAME COLUMNS BASED ON ELEMENT NAME
columns = {'input.composition1': 'Cr', 'input.composition2': 'Co', 'input.composition3': 'Cu', 'input.composition4': 'Fe', 'input.composition5': 'Ni'}
complete_dataframe.rename(columns = columns, inplace = True)
final_df = complete_dataframe.copy()

#USE ONLY DATA WITH CONVERGED AND COEXISTED SIMULATIONS
complete_dataframe = complete_dataframe.loc[(complete_dataframe['output.Coexistence'] == 1) & (complete_dataframe['output.Converged'] == 1)]

#DROP DUPLICATES, CONTAINING UNIQUE VALUES FOR EACH COMPOSITION
complete_dataframe = complete_dataframe.drop_duplicates(subset=['Cr', 'Co', 'Cu', 'Fe', 'Ni'])

#LIST OF DATES FOR EACH COMPOSITION RAN
df = pd.read_csv('cleaned_list.txt', sep=' ', header=None)
df[8] = [x.replace("_", "/") for x in df[8]]
runs_file = complete_dataframe.copy()

In [ ]:
#ADD MONTH, DAY, YEAR TO EACH VALUE
comparison = runs_file[runs_file['squid'].isin(df[8])]
df = df.rename({8 : 'squid'}, axis = 1)
merged_df = pd.merge(comparison, df[[5, 6, 7, 'squid']], left_on='squid', right_on='squid', how='left')

# Drop the 'squid' column from the merged DataFrame as it's no longer needed
merged_df.drop('squid', axis=1, inplace=True)
merged_df.drop('date', axis=1, inplace=True)

# Rename the columns 5, 6, 7, to month, day, year
merged_df.rename(columns={5: 'Month', 6: 'Day', 7: 'Year'}, inplace=True)

#Drop duplicates
merged_df = merged_df.drop_duplicates(subset=['Cr', 'Co', 'Cu', 'Fe', 'Ni'])

In [ ]:
def filter_dates(df, month, day, year):
    target_date = datetime(year, month, day)
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    filtered_df = df[df['Date'] <= target_date]
    return filtered_df
def convert_abbr_to_num(df, column_name):
    month_mapping = {'Jan':  1, 'Feb':  2, 'Mar':  3, 'Apr':  4, 'May':  5, 'Jun':  6,
                     'Jul':  7, 'Aug':  8, 'Sep':  9, 'Oct':  10, 'Nov':  11, 'Dec':  12}
    df[column_name] = df[column_name].apply(lambda x: month_mapping.get(x, x))
    
    return df

In [ ]:
#RENAME AND FILTER BY DATE
prior_to_2022_data = convert_abbr_to_num(merged_df, 'Month')
prior_to_2022_data = filter_dates(prior_to_2022_data, 1, 1, 2022)
prior_to_2022_data = prior_to_2022_data.reset_index()

In [ ]:
#SUBSET RELEVANT COLUMNS
columns = ['Cr', 'Co', 'Cu', 'Fe', 'Ni', 'input.Tsolid', 'input.Tliquid', 'output.melting_temperature']
known = prior_to_2022_data[columns].copy()

In [ ]:
display(known)

In [ ]:
sample = ['Cr', 'Co', 'Cu', 'Fe', 'Ni']

In [ ]:
#RULE OF MIXTURES FEATURE CALCULATION

data = {
  "Cr": [2.790000e+02, 1.400000e+00, 1.270000e-07, 4.900000e-06, 1.120000e+03, 2.944000e+03, 5.199610e+01, 2.100000e-01, 7.140000e+03, 1.660000e+00, 2.180000e+03],
  "Co": [2.090000e+02, 1.350000e+00, 6.000000e-08, 1.300000e-05, 7.000000e+02, 3.200000e+03, 5.893319e+01, 3.100000e-01, 8.900000e+03, 1.880000e+00, 1.768000e+03],
  "Cu": [1.300000e+02, 1.350000e+00, 1.720000e-08, 1.650000e-05, 8.740000e+02, 3.200000e+03, 6.354600e+01, 3.400000e-01, 8.920000e+03, 1.900000e+00, 1.357770e+03],
  "Fe": [2.110000e+02, 1.400000e+00, 1.000000e-07, 1.180000e-05, 4.900000e+02, 3.134000e+03, 5.584500e+01, 2.900000e-01, 7.874000e+03, 1.830000e+00, 1.811000e+03],
  "Ni": [2.000000e+02, 1.350000e+00, 7.200000e-08, 1.340000e-05, 7.000000e+02, 3.186000e+03, 5.869340e+01, 3.100000e-01, 8.908000e+03, 1.910000e+00, 1.728000e+03]
}

index = ['youngs_modulus', 'atomic_radius', 'electrical_resistivity', 'CTE', 'hardness', 'boiling_point', 'atomic_mass', 'poissons_ratio', 'density_of_solid', 'en_gosh', 'melting_point']

c_df = pd.DataFrame(data, index=index)

actual_qued_values = ['youngs_modulus', 'atomic_radius', 'electrical_resistivity','CTE','hardness','boiling_point',
                         'atomic_mass','poissons_ratio','density_of_solid','en_gosh','melting_point']


#Apply Rule of Mixtures
def rule_mixtures(df2,df,sample): 
    #Creats column of first rule mixture descriptors to concatenate to
    cr = 0
    for comps in sample:
        cr = cr+df2[comps]*df[comps][0]
    cr = cr.to_frame()

    #Concatenate rest of compositions with rule of mixtures applied
    for x in range(1,len(actual_qued_values)):
        cf = 0
        for comps in sample:
            cf = cf + df2[comps]*df[comps][x]
        cf.to_frame()
        cr = pd.concat([cr,cf], axis=1)

    #Renames columns to descriptors
    cr.columns = actual_qued_values
    return cr

In [ ]:
#ADD FEATURES
kn = rule_mixtures(known, c_df, sample)
known = pd.concat([known, kn], axis=1)

In [ ]:
#REARRANGE COLUMNS
column_to_move = known.pop('input.Tsolid')
col2_to_move = known.pop('input.Tliquid')
col3_to_move = known.pop('output.melting_temperature')
known['input.Tsolid'] = column_to_move
known['input.Tliquid'] = col2_to_move
known['output.melting_temperature'] = col3_to_move

In [ ]:
display(known)

# Melting Temperature vs Tsolid Temperature vs Tliquid Temperature Plots and Lines of Best Fits

Calculated and fitted lines of best fits based on the relationships between the 3 variables

In [ ]:
# RULE OF MIXTURES CALCULATED MELTING TEMPERATURE VS MOLECULAR DYNAMICS CALCULATED MELTING TEMPERATURE

plt.figure(figsize=(9, 9))

# Extract data from the DataFrame (relevant columns)
x = known['output.melting_temperature'].values
y = known['melting_point'].values

# Calculate the line of best fit
slope, intercept = np.polyfit(x, y, 1)

# Plot Formatting
plt.scatter(x, y, c='orange', alpha=0.7, s=100)
plt.plot(x, slope*x+intercept, color='red', linestyle='--', linewidth=2)
plt.grid(True)
plt.xlabel('Melting Temperature (K)', fontsize=23)
plt.ylabel('Rule Of Mixtures Calculated Temperature (K)', fontsize=23)
plt.tick_params(axis='both', labelsize=23)

# Calculate the equation of the line of best fit
equation = f'y = {intercept:.2f} + {slope:.2f}x'

#Equation Label
plt.text(0.02, 0.98, equation, transform=plt.gca().transAxes, fontsize=22, ha='left', va='top', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))

plt.show()

In [ ]:
print(slope, intercept)

In [ ]:
# Tsolid Temperature vs Molecular Dynamics Calculated Melting Temperature

plt.figure(figsize=(9, 9))

# Extract data from the DataFrame (relevant columns)
x = known['output.melting_temperature'].values
y = known['input.Tsolid'].values

# Calculate the line of best fit
slope, intercept = np.polyfit(x, y, 1)

# Plot Formatting
plt.scatter(x, y, c='orange', alpha=0.7, s=100)
plt.plot(x, slope*x+intercept, color='red', linestyle='--', linewidth=2)
plt.grid(True)
common_fontsize = 23
plt.xlabel('Melting Temperature (K)', fontsize=common_fontsize)
plt.ylabel('Tsolid Temperature (K)', fontsize=common_fontsize)
plt.tick_params(axis='both', labelsize=common_fontsize)

# Calculate the equation of the line of best fit
equation = f'y = {intercept:.2f} + {slope:.2f}x'

# Equation Label
plt.text(0.02, 0.98, equation, transform=plt.gca().transAxes, fontsize=common_fontsize, ha='left', va='top', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))

plt.show()


In [ ]:
print(slope)
print(intercept)

In [ ]:
# Tsolid Temperature vs Molecular Dynamics Calculated Melting Temperature

plt.figure(figsize=(9, 9))

# Extract data from the DataFrame
x = known['output.melting_temperature'].values
y = known['input.Tliquid'].values

# Calculate the line of best fit
slope, intercept = np.polyfit(x, y, 1)

#Plot Formatting
plt.scatter(x, y, c='orange', alpha=0.7, s=100)
plt.plot(x, slope*x+intercept, color='red', linestyle='--', linewidth=2)
plt.grid(True)
common_fontsize = 23
plt.xlabel('Melting Temperature (K)', fontsize=common_fontsize)
plt.ylabel('Tliquid Temperature (K)', fontsize=common_fontsize)
plt.tick_params(axis='both', labelsize=common_fontsize)

# Calculate the equation of the line of best fit
equation = f'y = {intercept:.2f} + {slope:.2f}x'

# Equation Label
plt.text(0.02, 0.98, equation, transform=plt.gca().transAxes, fontsize=common_fontsize, ha='left', va='top', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.3'))

plt.show()

In [ ]:
print(slope)
print(intercept)

 # Random Forest

In [ ]:
X = known.iloc[:, :-1]  # Features (all columns except the last one)
y = known.iloc[:, -1]   # Target (Melting Temperature)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Extract data from the DataFrame
x = y_train.values
y = X_train['input.Tsolid'].values

# Calculate the line of best fit (from training data only, to avoid contamination/bias)
slope, intercept = np.polyfit(x, y, 1)

In [ ]:
tsol_slope = slope
tsol_intercept = intercept

In [ ]:
# Extract data from the DataFrame
x = y_train.values
y = X_train['input.Tliquid'].values

# Calculate the line of best fit
slope, intercept = np.polyfit(x, y, 1)

In [ ]:
tliq_slope = slope
tliq_intercept = intercept

In [ ]:
X_train.drop(columns = ['input.Tsolid', 'input.Tliquid'], axis = 1, inplace = True)
X_test.drop(columns = ['input.Tsolid', 'input.Tliquid'], axis = 1, inplace = True)

In [ ]:
X_test.reset_index(inplace = True)

In [ ]:
X_test.drop(columns = ['index'], axis = 1, inplace = True)

In [ ]:
X_test.to_csv('rf1_test_inputs.csv', index=False)
X_train.to_csv('rf1_train_inputs.csv', index=False)
y_test.to_csv('rf1_test_outputs.csv', index=False)
y_train.to_csv('rf1_train_outputs.csv', index=False)

In [ ]:
#Initialize the Random Forest Model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

In [ ]:
# Calculate predictions for the training data
predictions_train = model.predict(X_train)

# Calculate predictions for the test data
predictions = model.predict(X_test)

#Predicted vs Actual Data for training and testing sets, respectively
a = y_train
a2 = y_test
b = predictions_train
b2 = predictions

#Plot Formatting
plt.figure(figsize=(9, 9))
plt.scatter(a, b, c='blue', label='Train Data', alpha=0.7, s=100)
plt.scatter(a2, b2, c='orange', label='Test Data', alpha=0.7, s=100)
plt.axline((0, 0), slope=1, color='black', linestyle='dashed', linewidth=1)
plt.xlim(1500, 2600)
plt.ylim(1500, 2600)
plt.tick_params(axis='both', labelsize=20) 
plt.xlabel('Molecular Dynamics', fontsize=20)
plt.ylabel('Random Forest', fontsize=20)
plt.title('Melting Temperature (K)', fontsize=20)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
test_dataset = pd.read_csv('rf1_test_inputs.csv')
data = test_dataset

In [ ]:
numpy_array = data.values

# Convert the NumPy array to a PyTorch tensor
data = torch.tensor(numpy_array, dtype=torch.float32)

# Test Set Evaluation

Evaluating the Random Forest on the test set, determining the new average number of simulations per alloy

In [ ]:
inputs = getSimToolInputs(MeltHEA)
#print(inputs)

In [ ]:
i = 0
for index in data:   
    inputs.time.value = 100 # -> number of picoseconds to run
    inputs.composition1.value = index[0].item()
    inputs.composition2.value = index[1].item()
    inputs.composition3.value = index[2].item()
    inputs.composition4.value = index[3].item()
    inputs.composition5.value = index[4].item()
    #no need to modify
    inputs.box_length.value = 18
    
    
    reshaped_data = index.reshape(1, -1)
    
    prediction = model.predict(reshaped_data).squeeze()
    
    inputs.Tsolid.value = (tsol_slope * prediction) + tsol_intercept
    inputs.Tliquid.value = (tliq_slope * prediction) + tliq_intercept
    r = Run(MeltHEA, inputs) # r is an object -> make list of ex. s.append(r)
    
    dataframe = r.getResultSummary()
    k = dataframe[['name', 'data']]
    k = k.transpose()
    k.columns = k.iloc[0]
    k = k.iloc[1:].reset_index(drop=True)
    k = k.drop('final_snapshot', axis = 1)
    k.to_csv('Alloy Data 5/alloy' + str(i) + '.csv', index=False, float_format='%.6f')
    coexist = bool(r.read('Coexistence'))
    conv = bool(r.read('Converged'))
    comp = 2
    print('First Done')
    
    while ((coexist and conv) is False):

        print('While Loop')
        
        
        inputs.time.value = 100 # -> number of picoseconds to run
        inputs.composition1.value = index[0].item()
        inputs.composition2.value = index[1].item()
        inputs.composition3.value = index[2].item()
        inputs.composition4.value = index[3].item()
        inputs.composition5.value = index[4].item()
        #no need to modify
        inputs.box_length.value = 18

       
        if ((coexist) and (not conv)):
            inputs.time.value = inputs.time.value + 50
        else:
            if (float(r.read('fraction_solid')) >= float(r.read('fraction_liquid'))):
                prediction = prediction * 1.025
            else:
                prediction = prediction * 0.975
    
        inputs.Tsolid.value = (tsol_slope * prediction) + tsol_intercept
        inputs.Tliquid.value = (tliq_slope * prediction) + tliq_intercept
        r = Run(MeltHEA, inputs) # r is an object -> make list of ex. s.append(r)

        dataframe = r.getResultSummary()
        k = dataframe[['name', 'data']]
        k = k.transpose()
        k.columns = k.iloc[0]
        k = k.iloc[1:].reset_index(drop=True)
        k = k.drop('final_snapshot', axis = 1)
        k.to_csv('Alloy Data 5/alloy' + str(i) + '_' + str(comp) + '.csv', index=False, float_format='%.6f')
        comp += 1
        coexist = bool(r.read('Coexistence'))
        conv = bool(r.read('Converged'))

    
    print("done with {}", i)
    i = i + 1

# Random Forest Evalution

Evaluating the Random Forest's reduction in average simulations

In [ ]:
#Reading a list of values processed before simulations ran
df = pd.read_csv('cleaned_list.txt', sep=' ', header=None)
df[8] = [x.replace("_", "/") for x in df[8]]
runs_file = final_df.copy()

#Keep only data that's present in df (matches by sqUID)
prior_to_2024_data = runs_file[runs_file['squid'].isin(df[8])]
df = prior_to_2024_data.copy()

In [ ]:
#Processing number of simulations per composition for data: new sims ran
cols = ['Cr', 'Co', 'Cu', 'Fe', 'Ni']

filtered_df = pd.DataFrame()

k = 0

for index in range(54): 
    counter = 1
    v = 2
    file_path = 'Alloy Data 5/alloy' + str(index) + '.csv'
    k = pd.read_csv(file_path)
    k.rename(columns = {'true_comp1': 'Cr', 'true_comp2': 'Co', 'true_comp3': 'Cu', 'true_comp4': 'Fe', 'true_comp5': 'Ni'}, inplace = True)
    
    cr = round(k['Cr'].item(), 1)
    co = round(k['Co'].item(), 1)
    cu = round(k['Cu'].item(), 1)
    fe = round(k['Fe'].item(), 1)
    ni = round(k['Ni'].item(), 1)
    
    temp_df = df[(df['Cr'] == cr) & (df['Co'] == co) & (df['Cu'] == cu) & (df['Fe'] == fe) & (df['Ni'] == ni)]
    
    
    filtered_df = pd.concat([filtered_df, temp_df], ignore_index=True)

In [ ]:
df = filtered_df

In [ ]:
#Grouping elemenets by composition
cols = ['Cr', 'Co', 'Cu', 'Fe', 'Ni']
groups = df.groupby(by = cols)
list_of_averages = []
unconverged_alloys = []

In [ ]:
#Constructing list of avg number of sims till convergence for each group/composition
for i, g in groups:
    if (len(g[(g['output.Coexistence'] == 1) & (g['output.Converged'] == 1)]) != 0):
        average = len(g)/len(g[(g['output.Coexistence'] == 1) & (g['output.Converged'] == 1)])
        list_of_averages.append(average)
    else:
        unconverged_alloys.append(i)

In [ ]:
#Processing number of simulations per composition for data: new sims ran
list_av = []
for index in range(54): 
    counter = 1
    v = 2
    file_path = 'Alloy Data 5/alloy' + str(index) + '_' + str(v) + '.csv'
    while (os.path.exists(file_path)):
        counter += 1
        v += 1
        file_path = 'Alloy Data 5/alloy' + str(index) + '_' + str(v) + '.csv'
    list_av.append(counter)

In [ ]:
values_list1 = list_of_averages  # Make sure this variable is correctly defined
values_list2 = list_av  # Ensure this variable is correctly defined

bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

binned_values1 = pd.cut(values_list1, bins=bins)
bin_counts1 = binned_values1.value_counts().sort_index()

binned_values2 = pd.cut(values_list2, bins=bins)
bin_counts2 = binned_values2.value_counts().sort_index()

total_counts1 = bin_counts1.sum()
total_counts2 = bin_counts2.sum()

# Change to count-based values
counts1 = bin_counts1.values
counts2 = bin_counts2.values

bin_labels = bin_counts1.index.astype(str)

plt.figure(figsize=(15, 9))
plt.bar(bin_labels, counts1, alpha=0.5, label='Domain Knowledge Initial Guesses', color = '#DC143C')
plt.bar(bin_labels, counts2, alpha=0.5, label='Data-Driven Initial Guesses', color='#004953')

plt.tick_params(axis='both', labelsize=18)

# Reintroduce the mean lines
mean_value1 = sum(values_list1) / len(values_list1)
mean_value2 = sum(values_list2) / len(values_list2)

# Draw vertical lines at the mean values
plt.axvline(x=mean_value1, color='#DC143C', linestyle='--', label='Mean: Farache et. al. Sims {:.2f}'.format(mean_value1))
plt.axvline(x=mean_value2, color='#004953', linestyle='--', label='Mean: RF-Sims {:.2f}'.format(mean_value2))

plt.xlabel('Average Number of Simulations per Alloy', fontsize = 20)
plt.ylabel('Number of Alloys', fontsize = 20)
plt.xticks(rotation=45)
plt.legend(fontsize = 16)

plt.show()